# Hands On with the Time Space Map

Time Space Map allows efficiently representing objects in a spatial map and some number of past spatial maps, so that opencog can access and process such data with a good compute performance. This kind of storage is a kind of probabilistic point cloud information over a period of time. We use octomap library and add a time buffer with relationship and access query functions exposed for programmers to store and process such data. this hans on assumes you have basic understanding of how to use cogserver from command line and are familiar with some basics of opencog along familiarity with scheme (guile) [Note: Working demo can be seen in ros-behavior-scripting git repository in file "src/time-map.scm"]

First of all, tell guile where it can find opencog stuff. You might want to add the following line to your .guile so it will do it at startup.

In [1]:
(add-to-load-path "/usr/local/share/opencog/scm")

Now load all required modules.

In [2]:
(use-modules (opencog) (opencog atom-types)(opencog pointmem) (opencog spacetime))

Now we can create a map (named "MyMap"). The following command creates an opencog space-time map, for storing locations of objects. Our map will be update every 66 millisdeconds (about 15 Hz), and will buffer 150 frames (about 10 seconds), and work with a one centimeter resolution (the native coords of the map are meters).


In [3]:
(cog-pointmem-create-map (SpaceMapNode "MyMap") (ListLink (NumberNode 0.01) (NumberNode 66) (NumberNode 150)))

(SpaceMapNode "MyMap")


This should return (SpaceMapNode "MyMap").
Now we can either step time units automatically or manually. 

Automatic time steps can be toggled using:

- (cog-pointmem-auto-step-time-on (SpaceMapNode "MyMap"))
- (cog-pointmem-auto-step-time-off (SpaceMapNode "MyMap"))

In this tutorial we will do it manually using:

In [4]:
(cog-pointmem-step-time-unit (SpaceMapNode "MyMap"))


()

Manual time step has to be done each time we need to create a new discrete time unit to attach atoms. The following call will map the (NumberNode 10) to the coordinates (1, 1, 1)and the time unit we have just created in the previous command.

In [5]:
(cog-pointmem-map-atom (SpaceMapNode "MyMap") 
                       (NumberNode 10) 
                       (ListLink 
                           (NumberNode 1.0) 
                           (NumberNode 1.0) 
                           (NumberNode 1.0)))

(NumberNode "10.000000")


We can query for atoms using several functions like:

In [6]:
 (cog-pointmem-get-locs-of-atom (SpaceMapNode "MyMap") (NumberNode 10))

(SetLink
   (AtLocationLink
      (SpaceMapNode "MyMap")
      (ListLink
         (NumberNode "10.000000")
         (ListLink
            (NumberNode "1.005000")
            (NumberNode "1.005000")
            (NumberNode "1.005000")
         )
      )
   )
)


The output of the above command should be:

    (SetLink

     (AtLocationLink
   
      (SpaceMapNode "MyMap")
      (ListLink      
      
         (NumberNode "10.000000")
         
         (ListLink
            (NumberNode "1.005000")
            (NumberNode "1.005000")
            (NumberNode "1.005000")
         )
      )
   )
)

or try this command:



In [7]:
(cog-pointmem-get-elapsed-list (SpaceMapNode "MyMap") (NumberNode 10))

(SetLink
   (AtTimeLink
      (NumberNode "10.000000")
      (TimeNode "2018-10-05 12:00:03 771")
   )
)


The output of the above command should be:

(SetLink
   (AtTimeLink
      (NumberNode "10.000000")
      (TimeNode "2017-02-01 16:08:19 352")
   )
)

Of course the time node will not have the same datetime as specified above

To add an observation after the first time slot of 66 milliseconds elapses, we should time step again.

In [8]:
 (cog-pointmem-step-time-unit (SpaceMapNode "MyMap"))

()

Now, adding the same NumberNode to the map and querying for elapsed list gives:

In [9]:
(cog-pointmem-map-atom (SpaceMapNode "MyMap") 
                       (NumberNode 10) 
                       (ListLink 
                           (NumberNode 2.0) 
                           (NumberNode 1.0) 
                           (NumberNode 1.0)))
(cog-pointmem-get-elapsed-list (SpaceMapNode "MyMap") (NumberNode 10))


(SetLink
   (AtTimeLink
      (NumberNode "10.000000")
      (TimeNode "2018-10-05 12:00:03 837")
   )
   (AtTimeLink
      (NumberNode "10.000000")
      (TimeNode "2018-10-05 12:00:03 771")
   )
)


Now we see two "occurrences" of the tracked NumberNode at different timestamps. Now we can see how the position of the tracked NumberNode changed in time. The last argument (NumberNode n) in the calls below means we want to know the position of the tracked Node at the timestamp (NOW - n).

In [10]:
(cog-pointmem-get-past-locs-of-atom (SpaceMapNode "MyMap") 
                                    (NumberNode 10)
                                    (NumberNode 66))

(SetLink
   (AtLocationLink
      (SpaceMapNode "MyMap")
      (ListLink
         (NumberNode "10.000000")
         (ListLink
            (NumberNode "1.005000")
            (NumberNode "1.005000")
            (NumberNode "1.005000")
         )
      )
   )
)


In [11]:
(cog-pointmem-get-past-locs-of-atom (SpaceMapNode "MyMap") 
                                    (NumberNode 10)
                                    (NumberNode 0))

(SetLink
   (AtLocationLink
      (SpaceMapNode "MyMap")
      (ListLink
         (NumberNode "10.000000")
         (ListLink
            (NumberNode "2.005000")
            (NumberNode "1.005000")
            (NumberNode "1.005000")
         )
      )
   )
)


This output means that the tracked object [(NumberNode 10) in our tutorial] changed from position (1,1,1) to (2,1,1) in the last 60 miliseconds.